In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import re
from tqdm import tqdm
import numpy as np

In [2]:
headers= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [3]:
def search_theme(page) : 
    url = "https://finance.naver.com/sise/theme.nhn?&page="
    res=requests.get(url + "{}".format(page),headers=headers)
    res.raise_for_status()
    soup=BeautifulSoup(res.text,"lxml") 
    hh=soup.find("table", attrs={"class":"type_1 theme"})
    list = hh.find_all("td", attrs={"class" : "col_type1"})
        #테마 이름 및 링크 return
    #ex) [<td class="col_type1"><a href="/sise/sise_group_detail.nhn?type=theme&amp;no=408">마리화나(대마)</a></td>
    return list

In [4]:
def search_page(link) :
    url = link
    res=requests.get(url,headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    hh = soup.find_all("div", attrs={"class" : "name_area"})
    return hh

In [5]:
def search_A(x) :
    if(x[0] == 'A') :
        return x[:7]
    return 

In [6]:
item_info = pd.read_csv("2_iem_info.csv")
item_info = item_info['iem_cd'].astype('string')
item_info = pd.read_csv("2_iem_info.csv")
#item_info.iem_cd.map(lambda x:search_A(x))
sub = item_info['iem_cd'].astype('string')
sub = sub.map(lambda x:search_A(x))
item_info.set_index(sub, drop = True, inplace = True)
item_info.drop(['iem_cd'], axis =1, inplace= True)

In [7]:
item_info = item_info.reset_index().rename(columns={"index": "id"}).dropna(axis = 0)
item_info.set_index('iem_cd', drop = True, inplace = True)
#item_info.drop(['id'], axis = 1, inplace= True)

In [8]:
dictionary = {}

In [9]:
for i in tqdm(range(1,8)) :
    theme_list = search_theme(i)
    for j in theme_list :
        url =  "https://finance.naver.com" + j.find("a")['href']
        page = search_page(url)
        theme_name = j.find("a").get_text()
        item_info["{}".format(theme_name)] = 0
        dictionary[theme_name] = 1
        for k in page :
            code_unarranged = k.find("a")['href']
            code = "A" + code_unarranged.replace("/item/main.naver?code=", "")
            #print(code)
            if(code not in item_info.index) :
                #요즘 나온 주식들
                #item_info.loc[code] = 0
                continue
            item_info.loc[code, theme_name] = 1
            dictionary[theme_name] = dictionary[theme_name] + 1
 
 

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:50<00:00,  7.28s/it]


### 종목정보가 없는 새로 추가된 항목들

In [10]:
item_info[item_info['iem_eng_nm'] == 0].index

Index([], dtype='object', name='iem_cd')

In [11]:
item_info

,iem_eng_nm,iem_krl_nm,해운,항공/저가 항공사(LCC),캐릭터상품,케이블TV SO/MSO,K-뉴딜지수(게임),여행,코로나19(화이자),게임,...,종합상사,원자력발전,철강 중소형,강관업체(Steel pipe),전선,"주류업(주정, 에탄올 등)",해저터널(지하화/지하도로 등),자전거,조선,아스콘(아스팔트 콘크리트)
iem_cd,,,,,,,,,,,,,,,,,,,,,
A000060,Meritz Fire & Marine Insurance ...,메리츠화재해상보험보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A000120,CJ LOGISTICS ...,씨제이대한통운보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A000540,HeungkukFire&MarineInsurance ...,흥국화재해상보험보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00088K,HANWHA(3PB) ...,한화3우선주(신형) ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A000950,Chonbang ...,전방보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A336370,Doosan Solus ...,두산솔루스보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A337120,SAMSUNG KODEX Fn Multi Factor ETF ...,삼성 KODEX Fn멀티팩터증권상장지수투자신탁[주식] ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A353190,NH SPECIAL PURPOSE ACQUISITION 16 COMPANY ...,엔에이치기업인수목적16호 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 중간 점검 : 
## item_info는 각 종목 코드를 index로, column들은 테마들로, excel에 없는 종목 코드(165개)들은 이름없이 코드에만 추가

In [12]:
item_info.loc['A000060']

iem_eng_nm          Meritz Fire & Marine Insurance                ...
iem_krl_nm          메리츠화재해상보험보통주                                  ...
해운                                                                  0
항공/저가 항공사(LCC)                                                      0
캐릭터상품                                                               0
                                          ...                        
주류업(주정, 에탄올 등)                                                      0
해저터널(지하화/지하도로 등)                                                    0
자전거                                                                 0
조선                                                                  0
아스콘(아스팔트 콘크리트)                                                      0
Name: A000060, Length: 243, dtype: object

In [13]:
for i in item_info.loc['A000060'] :
    if(i == 1) :print(i)

1


# 시가총액 순위
## 1~10 : Big_Company
## 10~ 20 : Middle_Company

http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0302

In [14]:
def fill_A(x):
    temp = "A"
    if len(x) != 6:
        for i in range(len(x),6) :
            temp += "0"
    return temp + x

In [15]:
rank = pd.read_csv('data_1831_20210910.csv', encoding='CP949')
sub = rank['종목코드'].astype('string')
sub = sub.map(lambda x:fill_A(x))

In [16]:
rank.set_index(sub, drop = True, inplace = True)

In [17]:
rank.columns

Index(['순위', '종목코드', '종목명', '시장구분', '종가', '대비', '등락률', '거래량', '거래대금', '시가총액',
       '시가총액비중', '상장주식수'],
      dtype='object')

In [18]:
Big_Company = rank.query("순위 <= 10")
Middle_Company = rank.query("순위 > 10 and 순위 <= 50")

In [19]:
item_info['Big_Company'] = 0
item_info['Middle_Company'] = 0

In [20]:
Big_Code = Big_Company.index.tolist()
Middle_Code = Middle_Company.index.tolist()

In [21]:
dictionary['Big_Company'] = 0
dictionary['Middle_Company'] = 0

In [22]:
for i in Big_Code :
    item_info.loc[i, 'Big_Company'] = 1
    dictionary['Big_Company']  = dictionary['Big_Company'] + 1
    
for i in Middle_Code :
    item_info.loc[i, 'Middle_Company'] = 1
    dictionary['Middle_Company']  = dictionary['Middle_Company'] + 1
    

In [23]:
item_info.query('Big_Company == 1')

,iem_eng_nm,iem_krl_nm,해운,항공/저가 항공사(LCC),캐릭터상품,케이블TV SO/MSO,K-뉴딜지수(게임),여행,코로나19(화이자),게임,...,철강 중소형,강관업체(Steel pipe),전선,"주류업(주정, 에탄올 등)",해저터널(지하화/지하도로 등),자전거,조선,아스콘(아스팔트 콘크리트),Big_Company,Middle_Company
iem_cd,,,,,,,,,,,,,,,,,,,,,
A005930,SamsungElectronics ...,삼성전자보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A005935,SamsungElectronics(1P) ...,삼성전자1우선주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A051910,"LGCHEM,LTD ...",LG화학보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A012330,HyundaiMobis ...,현대모비스보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A055550,"ShinhanFinancialGroup Co.,Ltd. ...",신한금융지주회사보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A068270,Celltrion ...,셀트리온보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A035420,NAVER ...,NAVER보통주 ...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
A000660,SK hynix ...,에스케이하이닉스보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
A005380,HyundaiMotor ...,현대자동차보통주 ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
user_item = item_info.index.tolist()

In [25]:
items = pd.read_csv("items.csv", encoding='CP949')

In [26]:
items = items[items.iem_cd.isin(user_item) == True]

In [27]:
dictionary["국내"] = 1
dictionary["국외"] = 1
dictionary["보통주"] = 1
dictionary["종류주"] = 1

In [28]:
def Comp(x, theme) :
    if x == theme :
        dictionary[theme] = dictionary[theme] + 1
    return x == theme

In [29]:
tags = items.copy()
tags["국내"] = tags["국내/국외"].apply(lambda x : int(Comp(x, '국내')))
tags["국외"] = tags["국내/국외"].apply(lambda x : int(Comp(x, '국외')))
tags["보통주"] = tags["구분코드"].apply(lambda x : int(Comp(x, '보통주')))
tags["종류주"] = tags["구분코드"].apply(lambda x : int(Comp(x, '종류주')))
tags = tags[["iem_cd","국내","국외","보통주","종류주"]]

In [30]:
dictionary

{'해운': 6,
 '항공/저가 항공사(LCC)': 10,
 '캐릭터상품': 9,
 '케이블TV SO/MSO': 8,
 'K-뉴딜지수(게임)': 12,
 '여행': 10,
 '코로나19(화이자)': 8,
 '게임': 35,
 '모바일게임(스마트폰)': 43,
 '반도체 대표주(생산)': 4,
 '3D 낸드(NAND)': 21,
 'LED장비': 14,
 '우주항공산업': 22,
 '플렉서블 디스플레이': 25,
 '코로나19(렘데시비르)': 4,
 '코로나19(치료제/백신 개발 등)': 35,
 '보톡스(보툴리눔톡신)': 11,
 '밥솥': 6,
 '코로나19(진단키트)': 37,
 'K-뉴딜지수(인터넷)': 12,
 'RFID(NFC 등)': 18,
 '日 수출 규제(국산화 등)': 19,
 '코로나19(진단/치료제/백신 개발 등)': 73,
 '건설 중소형': 32,
 '시스템반도체': 37,
 '반도체 재료/부품': 48,
 '야놀자 관련주': 5,
 'SSD': 9,
 '3D 프린터': 14,
 '두나무 관련주': 12,
 '2020 상반기 신규상장': 18,
 '4차산업 수혜주': 48,
 '카지노': 9,
 '줄기세포': 24,
 '통신장비': 38,
 '반도체 장비': 67,
 '화학섬유': 9,
 '마리화나(대마)': 8,
 '코로나19(나파모스타트)': 16,
 '전자결제(전자화폐)': 16,
 '증권': 23,
 '코로나19(스푸트니크V)': 13,
 'LCD 부품/소재': 47,
 '드론(Drone)': 24,
 '아이폰': 24,
 'LCD장비': 46,
 'IT 대표주': 7,
 '일자리(취업)': 9,
 '크래프톤 관련주': 7,
 '항공기부품': 16,
 '2019 상반기 신규상장': 29,
 '2차전지(장비)': 22,
 '바이오시밀러(복제 바이오의약품)': 19,
 'OLED(유기 발광 다이오드)': 72,
 '전력저장장치(ESS)': 27,
 '토스 관련주': 7,
 '기업인수목적회사(SPAC)': 48,
 '갤럭시 부품주': 

In [31]:
item_info = pd.merge(item_info, tags, left_on='iem_cd', right_on='iem_cd', how='inner')

In [32]:
item_info.index = item_info.iem_cd
item_info.drop('iem_cd', axis=1, inplace= True)

In [33]:
total_count = len(item_info.index)

In [34]:
for each_genre in dictionary:
    dictionary[each_genre] = np.log10(total_count/dictionary[each_genre])
  
dictionary

{'해운': 2.64064704020671,
 '항공/저가 항공사(LCC)': 2.418798290590354,
 '캐릭터상품': 2.4645557811510286,
 '케이블TV SO/MSO': 2.51570830359841,
 'K-뉴딜지수(게임)': 2.3396170445427287,
 '여행': 2.418798290590354,
 '코로나19(화이자)': 2.51570830359841,
 '게임': 1.874730246240078,
 '모바일게임(스마트폰)': 1.7853298350107671,
 '반도체 대표주(생산)': 2.8167382992623913,
 '3D 낸드(NAND)': 2.0965789958564343,
 'LED장비': 2.2726702549121156,
 '우주항공산업': 2.076375609768147,
 '플렉서블 디스플레이': 2.020858281918316,
 '코로나19(렘데시비르)': 2.8167382992623913,
 '코로나19(치료제/백신 개발 등)': 1.874730246240078,
 '보톡스(보툴리눔톡신)': 2.3774056054321284,
 '밥솥': 2.64064704020671,
 '코로나19(진단키트)': 1.8505965665233586,
 'K-뉴딜지수(인터넷)': 2.3396170445427287,
 'RFID(NFC 등)': 2.1635257854870473,
 '日 수출 규제(국산화 등)': 2.1400446896375245,
 '코로나19(진단/치료제/백신 개발 등)': 1.5554754304698977,
 '건설 중소형': 1.9136483122704475,
 '시스템반도체': 1.8505965665233586,
 '반도체 재료/부품': 1.7375570532147664,
 '야놀자 관련주': 2.7198282862543346,
 'SSD': 2.4645557811510286,
 '3D 프린터': 2.2726702549121156,
 '두나무 관련주': 2.3396170445427287

In [35]:
key_list = list(dictionary.keys())

In [36]:
for i in key_list:
    temp = item_info[i]
    temp[temp == 1] = dictionary[i]
    item_info[i] = temp

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp == 1] = dictionary[i]
<ipython-input-36-408d230f5a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [37]:
item_info[item_info['Big_Company'] > 1]

,iem_eng_nm,iem_krl_nm,해운,항공/저가 항공사(LCC),캐릭터상품,케이블TV SO/MSO,K-뉴딜지수(게임),여행,코로나19(화이자),게임,...,해저터널(지하화/지하도로 등),자전거,조선,아스콘(아스팔트 콘크리트),Big_Company,Middle_Company,국내,국외,보통주,종류주
iem_cd,,,,,,,,,,,,,,,,,,,,,
A005930,SamsungElectronics ...,삼성전자보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A005935,SamsungElectronics(1P) ...,삼성전자1우선주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.000000,1.332438
A051910,"LGCHEM,LTD ...",LG화학보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A012330,HyundaiMobis ...,현대모비스보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A055550,"ShinhanFinancialGroup Co.,Ltd. ...",신한금융지주회사보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A068270,Celltrion ...,셀트리온보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A035420,NAVER ...,NAVER보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.87473,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A000660,SK hynix ...,에스케이하이닉스보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000
A005380,HyundaiMotor ...,현대자동차보통주 ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,2.418798,0.0,-0.000166,0,0.020337,0.000000


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return result_df

In [39]:
temp_info = item_info['iem_krl_nm']

In [40]:
item_info.drop(['iem_eng_nm', 'iem_krl_nm'], axis=1, inplace= True)

In [41]:
print(item_info.shape)
print(item_info.describe())

(2623, 247)
                해운  항공/저가 항공사(LCC)        캐릭터상품  케이블TV SO/MSO   K-뉴딜지수(게임)  \
count  2623.000000     2623.000000  2623.000000   2623.000000  2623.000000   
mean      0.005034        0.008299     0.007517      0.006714     0.009812   
std       0.115203        0.141468     0.135926      0.129811     0.151221   
min       0.000000        0.000000     0.000000      0.000000     0.000000   
25%       0.000000        0.000000     0.000000      0.000000     0.000000   
50%       0.000000        0.000000     0.000000      0.000000     0.000000   
75%       0.000000        0.000000     0.000000      0.000000     0.000000   
max       2.640647        2.418798     2.464556      2.515708     2.339617   

                여행   코로나19(화이자)           게임  모바일게임(스마트폰)  반도체 대표주(생산)  ...  \
count  2623.000000  2623.000000  2623.000000  2623.000000  2623.000000  ...   
mean      0.008299     0.006714     0.024301     0.028587     0.003222  ...   
std       0.141468     0.129811     0.212094    

In [42]:
cs_df = cos_sim_matrix(item_info, item_info)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2613,2614,2615,2616,2617,2618,2619,2620,2621,2622
iem_cd,,,,,,,,,,,,,,,,,,,,,
A000060,1.000000e+00,5.611446e-05,1.000000e+00,8.344607e-09,0.008252,4.128651e-05,5.167071e-05,8.158260e-05,8.304435e-05,1.087069e-04,...,0.008252,6.640014e-05,5.177091e-05,7.183951e-05,5.668042e-05,3.868864e-05,0.008252,4.429921e-05,5.097001e-05,1.111121e-04
A000120,5.611446e-05,1.000000e+00,5.611446e-05,6.876480e-09,0.006800,3.402267e-05,4.257991e-05,6.722918e-05,6.843375e-05,8.958127e-05,...,0.006800,5.471788e-05,4.266248e-05,5.920026e-05,4.670822e-05,3.188186e-05,0.006800,3.650533e-05,4.200249e-05,9.156338e-05
A000540,1.000000e+00,5.611446e-05,1.000000e+00,8.344607e-09,0.008252,4.128651e-05,5.167071e-05,8.158260e-05,8.304435e-05,1.087069e-04,...,0.008252,6.640014e-05,5.177091e-05,7.183951e-05,5.668042e-05,3.868864e-05,0.008252,4.429921e-05,5.097001e-05,1.111121e-04
A00088K,8.344607e-09,6.876480e-09,8.344607e-09,1.000000e+00,0.000001,5.059406e-09,6.331926e-09,9.997443e-09,1.017657e-08,1.332135e-08,...,0.000001,8.136926e-09,6.344205e-09,8.803488e-09,6.945835e-09,4.741053e-09,0.000001,5.428594e-09,6.246060e-09,1.361611e-08
A000950,8.251963e-03,6.800135e-03,8.251963e-03,1.011227e-06,1.000000,5.003235e-03,6.261627e-03,9.886448e-03,1.006359e-02,1.317345e-02,...,1.000000,8.046588e-03,6.273769e-03,8.705749e-03,6.868720e-03,4.688417e-03,1.000000,5.368324e-03,6.176714e-03,1.346494e-02


In [43]:
cs_df.columns = item_info.index.tolist()

In [45]:
for i in range(2623):
    cs_df.iloc[i, i] = -1

In [46]:
cs_df

,A000060,A000120,A000540,A00088K,A000950,A000990,A001260,A001270,A001340,A001360,...,A319640,A322180,A322780,A323210,A330590,A336370,A337120,A353190,A900340,A950140
iem_cd,,,,,,,,,,,,,,,,,,,,,
A000060,-1.000000e+00,5.611446e-05,1.000000e+00,8.344607e-09,0.008252,4.128651e-05,5.167071e-05,8.158260e-05,8.304435e-05,1.087069e-04,...,0.008252,6.640014e-05,5.177091e-05,7.183951e-05,5.668042e-05,3.868864e-05,0.008252,4.429921e-05,5.097001e-05,1.111121e-04
A000120,5.611446e-05,-1.000000e+00,5.611446e-05,6.876480e-09,0.006800,3.402267e-05,4.257991e-05,6.722918e-05,6.843375e-05,8.958127e-05,...,0.006800,5.471788e-05,4.266248e-05,5.920026e-05,4.670822e-05,3.188186e-05,0.006800,3.650533e-05,4.200249e-05,9.156338e-05
A000540,1.000000e+00,5.611446e-05,-1.000000e+00,8.344607e-09,0.008252,4.128651e-05,5.167071e-05,8.158260e-05,8.304435e-05,1.087069e-04,...,0.008252,6.640014e-05,5.177091e-05,7.183951e-05,5.668042e-05,3.868864e-05,0.008252,4.429921e-05,5.097001e-05,1.111121e-04
A00088K,8.344607e-09,6.876480e-09,8.344607e-09,-1.000000e+00,0.000001,5.059406e-09,6.331926e-09,9.997443e-09,1.017657e-08,1.332135e-08,...,0.000001,8.136926e-09,6.344205e-09,8.803488e-09,6.945835e-09,4.741053e-09,0.000001,5.428594e-09,6.246060e-09,1.361611e-08
A000950,8.251963e-03,6.800135e-03,8.251963e-03,1.011227e-06,-1.000000,5.003235e-03,6.261627e-03,9.886448e-03,1.006359e-02,1.317345e-02,...,1.000000,8.046588e-03,6.273769e-03,8.705749e-03,6.868720e-03,4.688417e-03,1.000000,5.368324e-03,6.176714e-03,1.346494e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A336370,3.868864e-05,3.188186e-05,3.868864e-05,4.741053e-09,0.004688,2.345725e-05,2.935712e-05,4.635179e-05,4.718229e-05,6.176264e-05,...,0.004688,3.772575e-05,2.941404e-05,4.081618e-05,3.220342e-05,-1.000000e+00,0.004688,2.516894e-05,2.895901e-05,6.312923e-05
A337120,8.251963e-03,6.800135e-03,8.251963e-03,1.011227e-06,1.000000,5.003235e-03,6.261627e-03,9.886448e-03,1.006359e-02,1.317345e-02,...,1.000000,8.046588e-03,6.273769e-03,8.705749e-03,6.868720e-03,4.688417e-03,-1.000000,5.368324e-03,6.176714e-03,1.346494e-02
A353190,4.429921e-05,3.650533e-05,4.429921e-05,5.428594e-09,0.005368,2.685899e-05,3.361444e-05,5.307366e-05,5.402460e-05,7.071937e-05,...,0.005368,4.319669e-05,2.458575e-01,3.411624e-01,3.687352e-05,2.516894e-05,0.005368,-1.000000e+00,3.315860e-05,7.228414e-05


In [47]:
def Most_related(Code, num = 5) :
    temp2 = cs_df[Code].sort_values(ascending=False).head(5)
    for i in temp2.index :
        temp3 = i[0]
        print(temp_info.loc[temp3])

In [48]:
Most_related('A000060')

현대해상화재보험보통주                                                 
흥국화재해상보험보통주                                                 
한화손해보험보통주                                                   
롯데손해보험보통주                                                   
DB손해보험보통주                                                   
